# Secuencia de procesamiento de ETL

En este cuaderno se detalla el paso a paso del ETL que se va a implementar. El ejercicio consiste en crear un pipeline simple que relaciona datos de dos fuentes diferentes. Para el ejemplo se emplean los datos de los archivos `Countries.csv` y `Countries_metadata.csv`.
Se crearán tres pasos (extraer, transformar y cargar)

En este cuaderno, los datos se leen desde los archivos .csv y se guarda el resultado como un archivo .csv también. Sin embargo, la idea del ETL se explora a profundidad en el script `etl_process.py` en donde los datos de salida se escriben en una base de datos.

## 1. Extracción de datos

En el primer paso se extraen los datos desde el archivo en formato CSV. Si bien es un paso sencillo, ya que pandas permite la lectura de un CSV, no siempre en un ETL la fase de extracción es sencilla. Por ejemplo, los datos pueden estar alojados en un servidor 'on premises', en una base de datos, en un datalake o data warehouse. Pueden estar distribuidos en archivos de diferentes formatos, y muchas otras formas de donde se deben sacar los datos

In [2]:
import pandas as pd


countries = pd.read_csv("dataset/countries.csv")
countries.head()

,Countries,Population
0,Austria,9006398
1,Bolivia,11673021
2,China,1439323776
3,Denmark,5792202
4,Egypt,102334404


In [3]:
countries_metadata = pd.read_csv("dataset/Countries_metadata.csv")
countries_metadata.head()

,country_names,Land_Area,Region
0,Greece,128900,Europe
1,China,9388211,Asia
2,Denmark,42430,Europe
3,Ethiopia,1000000,Africa
4,Egypt,995450,Africa


## 2. Transformación de los datos

En este paso se harán algunas transformaciones a ambos conjuntos de datos. En la etapa de transformación se deben hacer tantas operaciones como se requieran para procesar la información y entregarla como requieran las reglas de negocio

In [4]:
countries["Population" ] = countries["Population"] / 1000
countries = countries.rename(columns={'Population':'Population_per_k'})
countries

,Countries,Population_per_k
0,Austria,9006.398
1,Bolivia,11673.021
2,China,1439323.776
3,Denmark,5792.202
4,Egypt,102334.404
5,Ethiopia,114963.588
6,Finland,5540.720
7,France,65273.511
8,Germany,83783.942
9,Greece,10423.054


In [5]:
countries_metadata['Land_Area'] = countries_metadata['Land_Area']/1000
countries_metadata = countries_metadata.rename(columns={'Land_Area':'Land_Area_per_k'})
countries_metadata

,country_names,Land_Area_per_k,Region
0,Greece,128.900,Europe
1,China,9388.211,Asia
2,Denmark,42.430,Europe
3,Ethiopia,1000.000,Africa
4,Egypt,995.450,Africa
5,Bolivia,1083.300,South America
6,Austria,82.409,Europe
7,France,547.557,Europe
8,Germany,348.560,Europe
9,Finland,303.890,Europe


## 3. Fase de carga
En la fase de carga se entregan los archivos preprocesados al destino. Si bien en el destino se tiene un csv que es fácil de manipular gracias a pandas. El destino puede ser una base de datos relacional ( o no relacional) puede ser un datalake, un data warehouse, o cualquier otro medio de almacenamieto de datos

In [7]:
countries.to_csv('dataset/output/countries.csv')
countries_metadata.to_csv('dataset/output//countries_metadata.csv')